In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import re
import rushd as rd
import scipy as sp
import seaborn as sns

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.09.28_exp089.2'/'export'
plates = pd.DataFrame({
    'data_path': [base_path/'293T', base_path/'iPS11'],
    'yaml_path': [base_path/'wells.yaml']*2,
    'cell': ['293T', 'iPS11'],
})
output_path = rd.rootdir/'output'/'KL_exp089.2_viral-titer-calculation'
cache_path = output_path/'exp089.2_viral-titer.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mGL-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

### Calculate titer
`rd.flow.moi` takes:

1. A DataFrame with the following columns:

    - condition
    - replicate
    - starting_cell_count
    - scaling (dilution factor relative to max_virus)
    - max_virus

2. Information to gate infected cells

    - color_column_name (channel to gate on)
    - color_column_cutoff (gate)

3. Optional parameters

    - output_path (where to save the plots)
    - summary_method (mean/median of replicates)

In [ ]:
# Create columns for rd.flow.moi
data.loc[data['cell']=='293T', 'starting_cell_count'] = 2e4
data.loc[data['cell']=='iPS11', 'starting_cell_count'] = 3e4

data['condition'] = data['construct'] + '_' + data['biorep'].astype(str)

In [ ]:
# Draw gates on uninfected population
gates = {}
for cell in ['293T','iPS11']:
    for channel in channel_list:
        gates[(cell,channel)] = data.loc[(data['construct']=='UI') & (data['cell']==cell), channel].quantile(0.999)
display(gates)

In [ ]:
plot_df = data[(data['construct']=='UI')]
x = 'mGL-A'
y = 'mRuby2-A'
g = sns.displot(data=plot_df, x=x, y=y, col='cell', kind='kde',
                log_scale=True, hue='construct', 
                common_norm=False, fill=False, levels=7,)

for cell, ax in g.axes_dict.items():
    ax.axvline(gates[(cell,x)], color='black', zorder=0)
    ax.axhline(gates[(cell,y)], color='black', zorder=0)

In [ ]:
cell = '293T'
plot_df = data[(data['cell']==cell)].groupby('construct').sample(1000)

g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', col='construct', kind='kde',
                log_scale=True, common_norm=False, fill=False, levels=7,
                hue='construct', row='biorep')

for _, ax in g.axes_dict.items():
    ax.axvline(gates[(cell,x)], color='black', zorder=0)
    ax.axhline(gates[(cell,y)], color='black', zorder=0)

In [ ]:
cell = 'iPS11'
plot_df = data[(data['cell']==cell)].groupby('construct').sample(1000)

g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', col='construct', kind='kde',
                log_scale=True, common_norm=False, fill=False, levels=7,
                hue='construct', row='biorep')

for _, ax in g.axes_dict.items():
    ax.axvline(gates[(cell,x)], color='black', zorder=0)
    ax.axhline(gates[(cell,y)], color='black', zorder=0)

In [ ]:
# Categorize cells into quadrants based on two gates
# Possible values:
#   0 = double negative
#   1 = x-positive
#   2 = y-positive
#   3 = double positive
def get_quadrant(x,y,gate_x,gate_y):
    df_quad = pd.DataFrame()
    df_quad['x'] = x > gate_x
    df_quad['y'] = y > gate_y
    df_quad['quadrant'] = df_quad['x'].astype(int) + df_quad['y'].astype(int)*2
    return df_quad['quadrant']

x = 'mGL-A'
y = 'mRuby2-A'
data.loc[data['cell']=='293T', 'quadrant'] = get_quadrant(data[x], data[y], gates[('293T',x)], gates[('293T',x)])
data.loc[data['cell']=='iPS11', 'quadrant'] = get_quadrant(data[x], data[y], gates[('iPS11',x)], gates[('iPS11',x)])

In [ ]:
df_titer_293T = rd.flow.moi(data[data['cell']=='293T'], 'quadrant', 0.5)
df_titer_iPS11 = rd.flow.moi(data[data['cell']=='iPS11'], 'quadrant', 0.5)

In [ ]:
df_titer_293T['cell'] = '293T'
df_titer_iPS11['cell'] = 'iPS11'
df_titer = pd.concat([df_titer_iPS11, df_titer_293T])
df_titer.reset_index(inplace=True)
display(df_titer)
df_titer.to_csv(output_path/'titer_mGL-mR2.csv')